In [1]:
import os
import h5py
import numpy as np
import argparse
import cv2
from pathlib import Path

import natsort

## Config

In [3]:
parent_dir = Path("../ALOHATaskCompressedData/")  # ← CHANGE THIS

In [4]:
for dataset_folder in sorted(parent_dir.iterdir()):
    print(dataset_folder)

    # Using natsort to naturally sort the episode files
    sorted_files = natsort.natsorted(dataset_folder.glob("episode*.hdf5"))

    for hdf5_file in sorted_files:
        print(hdf5_file)
        break

..\ALOHATaskCompressedData\apply_tape_closed_box
..\ALOHATaskCompressedData\apply_tape_closed_box\episode_0.hdf5
..\ALOHATaskCompressedData\decompress_hdf5.ipynb
..\ALOHATaskCompressedData\pick_scraper_from_rack
..\ALOHATaskCompressedData\pick_scraper_from_rack\episode_0.hdf5
..\ALOHATaskCompressedData\usb_mark_003
..\ALOHATaskCompressedData\usb_mark_003\episode_0.hdf5


In [5]:
## CONSTANTS
CAMERA_NAMES = ["cam_high", "cam_left_wrist", "cam_low", "cam_right_wrist"]
DCAMERA_NAMES = ["dcam_high", "dcam_low"]

## Helper Function

In [7]:
def load_compressed_hdf5(dataset_path):
    if not os.path.isfile(dataset_path):
        print(f'Dataset does not exist at \n{dataset_path}\n')
        exit()
    with h5py.File(dataset_path, 'r') as root:
        is_sim = root.attrs['sim']
        qpos = root['/observations/qpos'][()]
        qvel = root['/observations/qvel'][()]
        effort = root['/observations/effort'][()]
        action = root['/action'][()]

        image_dict = dict()
        for cam_name in root[f'/observations/images/'].keys():
            # decode images
            emc_images = root[f'/observations/images/{cam_name}'][()]
            image_dict[cam_name] = list()
            for img in emc_images:
                decompressed_image = cv2.imdecode(img , 1)
                image_dict[cam_name].append(decompressed_image)
    return is_sim, qpos, qvel, effort, action, image_dict

In [8]:
def save_images_to_video(images, out_path, fps=30, is_depth=False):
    print(images.shape)
    if images.ndim == 4:  # [T, H, W, C]
        h, w = images.shape[1:3]
    else:  # [T, H, W]
        h, w = images.shape[1:3]
        images = np.expand_dims(images, -1)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(str(out_path), fourcc, fps, (w, h), isColor=True)

    for frame in images:
        if is_depth:
            frame = np.squeeze(frame)
            frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)
            frame = frame.astype(np.uint8)
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        else:
            frame = frame.astype(np.uint8)
        writer.write(frame)

    writer.release()

## Dataset Processing

In [10]:
# Parameters
fps = 30
is_depth = True

In [11]:
# Process each dataset folder
for dataset_folder in sorted(parent_dir.iterdir()):
    if dataset_folder.is_dir():  
        print(f"📦 Processing: {dataset_folder.name}")

        # Create necessary output directories specific to this dataset
        out_dir = Path(f"./rmb_dataset/{dataset_folder.name}")  

        # Process each .h5 file in the dataset folder
        for i, hdf5_file in enumerate(natsort.natsorted(dataset_folder.glob("episode*.hdf5"))):
            print(f"📦 Processing {hdf5_file.name}")
            episode_name = f"episode_{i:06d}.rmb"
            rmb_dir = out_dir / episode_name
            rmb_dir.mkdir(parents=True, exist_ok=True)

            # Load and decompress data
            is_sim, qpos, qvel, effort, action, image_dict = load_compressed_hdf5(hdf5_file)

            # Package image data in flat-key dictionary format
            arrays = {}
            for cam_name, frames in image_dict.items():
                key = f"observations/images/{cam_name}"
                arrays[key] = np.stack(frames, axis=0)  # Convert list to array

            # Handle image data
            for key in arrays:
                if key.startswith("observations/images/"):
                    is_depth = False  # your dataset only has RGB images
                    cam_name = key.split("/")[-1]  # e.g., cam_left_wrist
                    suffix = "rgb_image"
                    video_name = f"{cam_name}_{suffix}.rmb.mp4"
                    print(f"🎞️ Saving video: {video_name} from key: {key}")
                    save_images_to_video(arrays[key], rmb_dir / video_name, fps=fps, is_depth=is_depth)
                        
            print(f"✅ Done: {episode_name}")

📦 Processing: apply_tape_closed_box
📦 Processing episode_0.hdf5
🎞️ Saving video: cam_high_rgb_image.rmb.mp4 from key: observations/images/cam_high
(800, 480, 640, 3)
🎞️ Saving video: cam_left_wrist_rgb_image.rmb.mp4 from key: observations/images/cam_left_wrist
(800, 480, 640, 3)
🎞️ Saving video: cam_low_rgb_image.rmb.mp4 from key: observations/images/cam_low
(800, 480, 640, 3)
🎞️ Saving video: cam_right_wrist_rgb_image.rmb.mp4 from key: observations/images/cam_right_wrist
(800, 480, 640, 3)
✅ Done: episode_000000.rmb
📦 Processing episode_1.hdf5
🎞️ Saving video: cam_high_rgb_image.rmb.mp4 from key: observations/images/cam_high
(800, 480, 640, 3)
🎞️ Saving video: cam_left_wrist_rgb_image.rmb.mp4 from key: observations/images/cam_left_wrist
(800, 480, 640, 3)
🎞️ Saving video: cam_low_rgb_image.rmb.mp4 from key: observations/images/cam_low
(800, 480, 640, 3)
🎞️ Saving video: cam_right_wrist_rgb_image.rmb.mp4 from key: observations/images/cam_right_wrist
(800, 480, 640, 3)
✅ Done: episode_0

KeyboardInterrupt: 

---